In [1]:
import spacy
import pandas as pd
from tqdm import tqdm

## 初期化

In [2]:
keywords = set(["シャリ"])

def check_keyword(text : str):
    for keyword in keywords:
        if keyword in text:
            return True
    return False

In [3]:
stopwords = set([
    "シャリ",
    "お",
    "料理長",
    "　",
    "副",
    "いる",
    "する",
    "前",
    "くれる",
    "なる",
    "いう",
    "入れる",
    "ある",
    "同じ",
    "良い",
    "ない",
    "くる",
    "笑",
    "いい",
    "いく",
    "こと",
    "思う",
    "もの",
    "おる",
    "さん"
])

In [4]:
#例文（ChatGPTにより生成）
sample_text = "この度、特別な機会にある高級寿司店での食事を楽しむことができました。店内は落ち着いた雰囲気で、スタッフの対応も素晴らしかったです。メニューは多彩で、新鮮な魚介類の品揃えに感動しました。シャリは絶妙な締め加減で、口に入れると心地よい食感が広がります。寿司ネタとの相性も抜群で、一貫一貫がバランスの取れた美味しさでした。特におすすめは、トロやウニの握り寿司で、口の中でとろけるような味わいが堪能できました。もう一度シャリの話に戻りますが、シャリは、程よい酸味と口溶けの良さが特徴で、寿司ネタとのハーモニーが絶妙でした。口に入れるとほっと米がほころぶ。ここまでシャリが良い店は珍しい。価格は高めですが、その価値に見合う素晴らしい食体験でした。また訪れたいと思わせる高級寿司店でした。"

In [5]:
ginza = spacy.load("ja_ginza_electra")

c:\Users\brigh\anaconda3\envs\jpnlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## データ読み込み

In [6]:
with open("reviews.csv") as f:
    reviews = f.read().split("\n")
    
len(reviews)

5068

## 「シャリ」からの近さでスコア化して集計
  
文ごとにスコアを与える  
「シャリ」を含む文を１として、その近隣の文も（シャリの話をしている可能性が高いので）ポワソン分布的にスコアを与える  
文内の単語はその文のスコアだけ加点される

In [8]:
#原文を文ごとにわけ、さらに形態素にわける
def breakdown(text):
    _text = text.replace("<br>", "\n")
    sents = ginza(_text).sents
    
    sents_text = []
    sents_tokenized = []
    for sent in sents:
        sents_text.append(str(sent))
        sents_tokenized.append([])
        for token in sent:
            sents_tokenized[-1].append((token.lemma_, token.pos_))
            
    return sents_text, sents_tokenized

sample_sents, sample_tokens = breakdown(sample_text)

print(sample_sents, sample_tokens)

['この度、特別な機会にある高級寿司店での食事を楽しむことができました。', '店内は落ち着いた雰囲気で、スタッフの対応も素晴らしかったです。', 'メニューは多彩で、新鮮な魚介類の品揃えに感動しました。', 'シャリは絶妙な締め加減で、口に入れると心地よい食感が広がります。', '寿司ネタとの相性も抜群で、一貫一貫がバランスの取れた美味しさでした。', '特におすすめは、トロやウニの握り寿司で、口の中でとろけるような味わいが堪能できました。', 'もう一度シャリの話に戻りますが、シャリは、程よい酸味と口溶けの良さが特徴で、寿司ネタとのハーモニーが絶妙でした。', '口に入れるとほっと米がほころぶ。', 'ここまでシャリが良い店は珍しい。', '価格は高めですが、その価値に見合う素晴らしい食体験でした。', 'また訪れたいと思わせる高級寿司店でした。'] [[('この', 'DET'), ('度', 'NOUN'), ('、', 'PUNCT'), ('特別', 'ADJ'), ('だ', 'AUX'), ('機会', 'NOUN'), ('に', 'ADP'), ('ある', 'VERB'), ('高級', 'NOUN'), ('寿司店', 'NOUN'), ('で', 'ADP'), ('の', 'ADP'), ('食事', 'NOUN'), ('を', 'ADP'), ('楽しむ', 'VERB'), ('こと', 'NOUN'), ('が', 'ADP'), ('できる', 'AUX'), ('ます', 'AUX'), ('た', 'AUX'), ('。', 'PUNCT')], [('店内', 'NOUN'), ('は', 'ADP'), ('落ち着く', 'VERB'), ('た', 'AUX'), ('雰囲気', 'NOUN'), ('で', 'AUX'), ('、', 'PUNCT'), ('スタッフ', 'NOUN'), ('の', 'ADP'), ('対応', 'NOUN'), ('も', 'ADP'), ('素晴らしい', 'ADJ'), ('た', 'AUX'), ('です', 'AUX'), ('。', 'PUNCT')], [('メニュー', 'NOUN'), ('は', 'ADP'), ('多彩', 'ADJ'), ('だ', 'AUX

In [9]:
#文章ごとのスコアを産出
def evaluate_sentent_score(sents):
    has_keyword = []
    sents_n = 0
    for sent in sents:
        has_keyword.append(check_keyword(str(sent)))
        sents_n += 1
        
    scores = [0.0] * sents_n
    
    def put_score(score, ind):
        if (ind < 0) or (len(scores) <= ind):
            return
        elif scores[ind] < score:
            scores[ind] = score
    
    #近くにあるものを加点
    for cnt in range(sents_n):
        if has_keyword[cnt]:
            put_score(0.2, cnt-1)
            put_score(1.0, cnt)
            put_score(0.3, cnt+1)
            put_score(0.1, cnt+2)
            put_score(0.01, cnt+3)
            
    #キーワード持ち文の間にあるものをブースト
    BETWEEN_MAX = 2
    for between in range(2, BETWEEN_MAX+1):
        for cnt in range(sents_n-between):
            if has_keyword[cnt] and has_keyword[cnt+between]:
                for cnt1 in range(cnt, cnt+between):
                    if not has_keyword[cnt1]:
                        put_score(0.8, cnt1)
    
    return scores

sample_scores = evaluate_sentent_score(sample_sents)

for cnt in range(len(sample_sents)):
    print(sample_scores[cnt], sample_sents[cnt])

0.0 この度、特別な機会にある高級寿司店での食事を楽しむことができました。
0.0 店内は落ち着いた雰囲気で、スタッフの対応も素晴らしかったです。
0.2 メニューは多彩で、新鮮な魚介類の品揃えに感動しました。
1.0 シャリは絶妙な締め加減で、口に入れると心地よい食感が広がります。
0.3 寿司ネタとの相性も抜群で、一貫一貫がバランスの取れた美味しさでした。
0.2 特におすすめは、トロやウニの握り寿司で、口の中でとろけるような味わいが堪能できました。
1.0 もう一度シャリの話に戻りますが、シャリは、程よい酸味と口溶けの良さが特徴で、寿司ネタとのハーモニーが絶妙でした。
0.8 口に入れるとほっと米がほころぶ。
1.0 ここまでシャリが良い店は珍しい。
0.3 価格は高めですが、その価値に見合う素晴らしい食体験でした。
0.1 また訪れたいと思わせる高級寿司店でした。


In [10]:
scores = {}

def add_score(word, score):
    global scores
    
    if word in scores:
        scores[word] += score
    else:
        scores[word] = score

#単語にスコアを与える
def analyze(text):
    global scores
    
    if not check_keyword(text):
        return
    
    sents, tokens = breakdown(text)
    
    sents_scores = evaluate_sentent_score(sents)
    
    for cnt_sent in range(len(sents)):
        score = sents_scores[cnt_sent]
        used = set()
        for token in tokens[cnt_sent]:
            #品詞
            if (token[1] in ["VERB", "ADJ", "NOUN"]) and (token[0] not in stopwords) and (token[0] not in used):
                add_score(token[0], score)
                used.add(token[0])

#デモ
analyze(sample_text)
print(scores)

#デモデータをリセット
scores = {}

{'度': 0.0, '特別': 0.0, '機会': 0.0, '高級': 0.1, '寿司店': 0.1, '食事': 0.0, '楽しむ': 0.0, '店内': 0.0, '落ち着く': 0.0, '雰囲気': 0.0, 'スタッフ': 0.0, '対応': 0.0, '素晴らしい': 0.3, 'メニュー': 0.2, '多彩': 0.2, '新鮮': 0.2, '魚介類': 0.2, '品揃え': 0.2, '感動': 0.2, '絶妙': 2.0, '締め': 1.0, '加減': 1.0, '口': 3.0, '心地よい': 1.0, '食感': 1.0, '広がる': 1.0, '寿司ネタ': 1.3, '相性': 0.3, '抜群': 0.3, '一貫': 0.3, 'バランス': 0.3, '取れる': 0.3, '美味しさ': 0.3, 'おすすめ': 0.2, 'トロ': 0.2, 'ウニ': 0.2, '握り寿司': 0.2, '中': 0.2, 'とろける': 0.2, '味わい': 0.2, '堪能': 0.2, '一度': 1.0, '話': 1.0, '戻る': 1.0, '程よい': 1.0, '酸味': 1.0, '溶ける': 1.0, '特徴': 1.0, 'ハーモニー': 1.0, '米': 0.8, 'ほころぶ': 0.8, '店': 1.0, '珍しい': 1.0, '価格': 0.3, '高め': 0.3, '価値': 0.3, '見合う': 0.3, '食': 0.3, '体験': 0.3, '訪れる': 0.1}


In [11]:
#分析
scores = {}
for review in tqdm(reviews):
    analyze(review)


100%|██████████| 5068/5068 [1:14:39<00:00,  1.13it/s]


### 結果

In [13]:
#結果を表示
# 辞書の値を基準にソートして、上位100件を表示
sorted_items = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:100]

for key, value in sorted_items:
    print(f"word: {key}  score: {value}")

word: ネタ  score: 799.1900000000007
word: 美味しい  score: 422.2000000000008
word: 赤酢  score: 357.03000000000026
word: 握り  score: 331.89999999999947
word: 寿司  score: 280.1799999999994
word: 食べる  score: 279.8099999999998
word: 鮨  score: 202.80999999999966
word: 赤  score: 175.51999999999987
word: バランス  score: 171.75
word: 口  score: 169.0699999999998
word: 握る  score: 165.86999999999992
word: 中  score: 160.98999999999964
word: 味  score: 154.29999999999978
word: 温度  score: 153.51
word: 酢  score: 150.14999999999995
word: 最高  score: 149.01000000000002
word: 感  score: 146.2299999999999
word: 白  score: 134.31999999999994
word: 脂  score: 133.89000000000001
word: 酸味  score: 131.34999999999997
word: 合わせる  score: 130.67999999999998
word: 好み  score: 125.58999999999996
word: マグロ  score: 124.06999999999992
word: 合う  score: 123.35999999999996
word: 方  score: 120.54999999999993
word: 大きい  score: 118.29
word:    score: 117.87999999999985
word: 感じ  score: 116.91999999999996
word: 使う  score: 114.20999999999992
